In [1]:
from api_token import token
import todoist as td
import pandas as pd
import datetime as dt

In [2]:
api = td.TodoistAPI(token)
# Sync to API to get latest data
api.sync();

In [3]:
# when the api query has no items (tasks) to report, it returns a dictionary with two empty keys e.g.
# {'items':[], 'projects':[]}
# Therefore I need to create a function to check if the keys in the dictionary have values

def has_values(dictionary):
    
    # For each key in dictionary, check if keys in dict have values
    values = [any(dictionary[key]) for key in dictionary]
    
    # Create a unique list of values
    output = list(set(values))
    
    return output[0]

In [4]:
# The API has a maximum of 200 items that can be returned, use while loop to get all items

# Items are retrieved in descending order by date.
# offset indicates how many items should be skipped
completed_tasks_list = []
limit = 200
offset = 0
not_empty = True

while not_empty:
    # API call, retrieving between 0 and 200 items at a time
    completed_tasks = api.completed.get_all(limit=limit, offset=offset)
    not_empty = has_values(completed_tasks) # if false it will exit the loop
    completed_tasks_list.append(completed_tasks) # add items to list
    offset += limit # increase offset to get the next set of results
    

In [5]:
# Create a list of dfs for each result of items
completed_dfs = [pd.DataFrame(completed_tasks_list[i]['items']) for i in range(0,len(completed_tasks_list))]

In [6]:
# Combine dfs into one large df
df = pd.concat(completed_dfs)

In [7]:
# Check num of rows and columns
df.shape

(943, 7)

In [39]:
# df.head()

In [38]:
# TODO add project_name field from projects
# our df doesn't yet have project name
# create new dict of id and project name
projects = api.projects.all() # Get list of projects from api
td_projects_dict = {projects[i].data['name']:projects[i].data['id'] for i,data in enumerate(projects)}
# projects = {int(key):td_projects_dict[key]['name'] for key in td_projects_dict}
td_projects_dict

{'Inbox': 2240336392,
 '❶ This Week': 2240338076,
 '❷ Next Week': 2250738735,
 '❸ This Month': 2250738747,
 '❹ Next Month': 2250738761,
 '❺ Further Than Next Month': 2250741928,
 '🧠 Project Ideas': 2250742579,
 '🚧 Backlog 🚧': 2240553789,
 '⏱ Routines': 2240717407,
 '🥓 JET': 2240338162,
 '🏡 Reign Property': 2240484947}

In [ ]:
# map project names to df
# yday_completed_tasks_df['project_name'] = yday_completed_tasks_df['project_id'].map(projects)